# American Ninja Warrior Scraping and Visualization

## By: Jeff Hale

## Plan

### Scrape American Ninja Warrior (ANW) contestant data over several years, combine the data into a DataFrame, and do some exploratory visual analysis.

## Background
[American Ninja Warrior](https://www.nbc.com/american-ninja-warrior) is a televised competition in whih contestents try to complete a serious of obstacles as quickly as possible, without falling. My kids love watching the show. Who knew it's been nominated for 3 Emmys?

Ten-time ninja warrior Matt Laessig also happens to lead [data.world](https://data.world), which offers a platform with data sets and an interface for data analysis. The platform has a data set on the obstacles and a link to a [Fantasy American Ninja Warrior](http://www.anwfantasy.com/members/contestants/season10.php) website that provides info about contestant results. 

Data for seasons 1 through 10 is on the website. In seasons 4 through 10 the show followed a similar format. Those are also the most recent seasons, so we'll focus on those. 

Let's get to it!

## Set-up
Load the necessary libraries.
Configure the Jupyter Notebook settings.
Load the data into a pandas DataFrame.

In [1]:
# essentials
import numpy as np 
import pandas as pd 

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# scraping
import requests
from bs4 import BeautifulSoup as bs 

# reproducibility
np.random.seed(34)

# Jupyter magic
%reload_ext autoreload
%autoreload 2
%matplotlib inline

sns.set()

In [2]:
# !ls              # list the file in the working directory

In [3]:
# !pip list      # list the package version numbers for reproducibiity

We might get some predictive benefit if we could evaluate the affect of the completion rate of an obstacle has on its future inclusion. There is a site we could scrape that should give us that info (http://www.anwfantasy.com/members/contestants/season10.php).

## Scrape contestant obstacle failure data

Let's use Beautiful Soup with the requests package to scrape data from the anwfantasy.com site. 

We'll scrape with the lxml parser because it's faster than the default html parser. You'll need to have lxml installed to use that parser. See the [Beatiful Soup docs]( https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser).

In [4]:
result = requests.get("http://www.anwfantasy.com/members/contestants/season4.php")
c = result.content
soup = bs(c)

In [ ]:
soup.find("a")

<a class="navbar-brand" href="/" style="margin-top:0px; padding-top:0px; background:none;"><img class="img img-responsive" src="/images/img-logo.png" style="max-width:130px;"/></a>

Alright, we're in! We were able to scrape a link as a test. Now let's get some data. We need to find out what page element, css class, or id we can latch onto to retrieve the data we need. 

Inspecting the code with Chrome (right click and select *Inspect*) shows that we want table row data ('tr').

In [ ]:
table_rows = soup.find_all('tr')
table_rows[:5]

[<tr><th class="headerSlim" style="text-align:center" width="200"> </th>
 <th class="headerSlim" style="text-align:center" width="200"> </th>
 <th class="headerSlim" colspan="2" style="text-align:left;"><div style="border-bottom:solid 1px #307DBE; width:99%; padding-left:15px;">city qualifier</div></th>
 <th class="headerSlim" colspan="2" style="text-align:left; "><div style="border-bottom:solid 1px #307DBE; width:99%; padding-left:15px;">city final</div></th>
 <th class="headerSlim" colspan="2" style="text-align:left; "><div style="border-bottom:solid 1px #307DBE; width:99%; padding-left:15px;">stage 1</div></th>
 <th class="headerSlim" colspan="2" style="text-align:left; "><div style="border-bottom:solid 1px #307DBE; width:99%; padding-left:15px;">stage 2</div></th>
 <th class="headerSlim" colspan="2" style="text-align:left; "><div style="border-bottom:solid 1px #307DBE; width:99%; padding-left:15px;">stage 3</div></th>
 </tr>, <tr>
 <th class="header" style="width:250px">Name       

Let's loop through the table rows to get all the "td"s.

In [ ]:
data_4 = []

for tr in table_rows:
    td = tr.find_all( 'td')
    row = [tr.text for tr in td]
    data_4.append(row)
    
df_s4 = pd.DataFrame(data_4, columns=[
    'fantasy_points', 
    'city_q_fail', 
    'city_q_time',
    'city_f_fail', 
    'city_f_time',
    'stage1_fail', 
    'stage1_time',
    'stage2_fail', 
    'stage2_time',
    'stage3_fail',
])

Let's add a column for stage4 that occurs in some later years. Let's also add a column to keep track of the year.

In [ ]:
df_s4['stage4_fail'] = np.NaN
df_s4['year'] = 4

In [ ]:
df_s4.head()

,fantasy_points,city_q_fail,city_q_time,city_f_fail,city_f_time,stage1_fail,stage1_time,stage2_fail,stage2_time,stage3_fail,stage4_fail,year
0,None,None,None,None,None,None,None,None,None,None,NaN,4
1,None,None,None,None,None,None,None,None,None,None,NaN,4
2,27.5,complete (nw),58.55,complete (nw),163.7,Spinning Bridge,,,,,NaN,4
3,39,complete (ms),53.99,complete (ms),143.22,Complete,98.29,Metal Spin,,,NaN,4
4,39,complete (se),76.26,complete (se),132.44,Complete,101,Balance Tank,,,NaN,4


Let's select all the rows other than the first two rows that contained header info, because we made our own shortened column names already. Let's reset the index, so it starts with 0, too.

In [ ]:
df_s4 = df_s4.iloc[2:]
df_s4.reset_index(drop=True, inplace=True)
df_s4.head()

,fantasy_points,city_q_fail,city_q_time,city_f_fail,city_f_time,stage1_fail,stage1_time,stage2_fail,stage2_time,stage3_fail,stage4_fail,year
0,27.5,complete (nw),58.55,complete (nw),163.7,Spinning Bridge,,,,,NaN,4
1,39,complete (ms),53.99,complete (ms),143.22,Complete,98.29,Metal Spin,,,NaN,4
2,39,complete (se),76.26,complete (se),132.44,Complete,101,Balance Tank,,,NaN,4
3,51.5,complete (sw),50.15,complete (sw),198.03,Complete,91,Complete,104.97,Hang Climb,NaN,4
4,38,complete (se),62.24,complete (se),124.1,Complete,96.08,Unstable Bridge,,,NaN,4


Now let's scrape the contestant names. They are stored in "<th>" elements. We'll use a for loop to loop through the table rows again. We'll make a list of lists and append the contestant data to the *data_4_contestants* list.

Then, as a check, we'll display the first few rows to make sure things look as expected. 

In [ ]:
data_4_contestants = []

for tr in table_rows:
    th = tr.find_all('th')
    row = [tr.text for tr in th]
    data_4_contestants.append(row)

data_4_contestants[:5]

[['\xa0',
  '\xa0',
  'city qualifier',
  'city final',
  'stage 1',
  'stage 2',
  'stage 3'],
 ['Name\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0',
  'Fantasy Points',
  'Obstacle Failed',
  'Completion Time(s)',
  'Obstacle Failed',
  'Completion Time(s)',
  'Obstacle Failed',
  'Completion Time(s)',
  'Obstacle Failed',
  'Completion Time(s)',
  'Obstacle Failed'],
 ['David Campbell'],
 ['Paul Kasemir'],
 ['Travis Rosen']]

Alrighty, looking good. Now let's drop the header columns and make a DataFrame that excludes the first two lists with header data.

In [ ]:
df_s4_contestants = pd.DataFrame(data_4_contestants[2:], columns=['contestant'])
df_s4_contestants.head()

,contestant
0,David Campbell
1,Paul Kasemir
2,Travis Rosen
3,Brent Steffensen
4,Drew Drechsel


Nice! Now we have just the contestants. Let's combine the DataFrames to reunite the contestants' names with their performance.

In [ ]:
df_4_combo = df_s4.join(df_s4_contestants, how = "outer")
df_4_combo.head()

,fantasy_points,city_q_fail,city_q_time,city_f_fail,city_f_time,stage1_fail,stage1_time,stage2_fail,stage2_time,stage3_fail,stage4_fail,year,contestant
0,27.5,complete (nw),58.55,complete (nw),163.7,Spinning Bridge,,,,,NaN,4,David Campbell
1,39,complete (ms),53.99,complete (ms),143.22,Complete,98.29,Metal Spin,,,NaN,4,Paul Kasemir
2,39,complete (se),76.26,complete (se),132.44,Complete,101,Balance Tank,,,NaN,4,Travis Rosen
3,51.5,complete (sw),50.15,complete (sw),198.03,Complete,91,Complete,104.97,Hang Climb,NaN,4,Brent Steffensen
4,38,complete (se),62.24,complete (se),124.1,Complete,96.08,Unstable Bridge,,,NaN,4,Drew Drechsel


In [ ]:
df_4_combo.tail()

,fantasy_points,city_q_fail,city_q_time,city_f_fail,city_f_time,stage1_fail,stage1_time,stage2_fail,stage2_time,stage3_fail,stage4_fail,year,contestant
278,0,Quad Steps (mw),,,,,,,,,NaN,4,Jacob Reeves
279,0,Quad Steps (mw),,,,,,,,,NaN,4,Doug Smith
280,0,Quad Steps (se),,,,,,,,,NaN,4,Daniel Vanegas
281,0,Quad Steps (se),,,,,,,,,NaN,4,Cristille Boyd
282,0,Quad Steps (ne),,,,,,,,,NaN,4,Bill Schlagter


Ok. Looks good. Let's name our main DataFrame "df" for ease of use.

In [ ]:
df = df_4_combo

To get the data for years 5 through 10, let's make a loop that scrapes each year page and appends it to the DataFrame.

In Seasons 7-10 there is an extra column - "Stage4". We'll account for that as we're building our DataFrames.

These are the fun kinds of inconsistencies in data that come up in real-world scraping problems. 

The cell below could take a few minutes to run as it scrapes the data.

We'll also add the year as a column value.

In [ ]:
def scrape_year(year):
    '''take the year as an integer and return the webpage data for ANW for that year'''
    result = requests.get(f"http://www.anwfantasy.com/members/contestants/season{year}.php")
    c = result.content
    soup = bs(c, "lxml")
    return soup

    
for year in range(5,11):
    soup = scrape_year(year)
    data = []
    data_contestants = []
    
    table_rows = soup.find_all('tr')

    for tr in table_rows:
        
        # find the row data other than the contestant names
        td = tr.find_all( 'td')
        row = [tr.text for tr in td]
        data.append(row)
        
        # find the contestant names
        th = tr.find_all('th')
        c_row = [tr.text for tr in th]
        data_contestants.append(c_row)
    
    # make DataFrame that excludes first two rows
    
    if year < 7:
        df_anw = pd.DataFrame(data[2:], columns=[
            'fantasy_points', 
            'city_q_fail', 
            'city_q_time',
            'city_f_fail', 
            'city_f_time',
            'stage1_fail', 
            'stage1_time',
            'stage2_fail', 
            'stage2_time',
            'stage3_fail',
            ])
        
        # add stage4 column because not present those years
        df_anw['stage4_fail'] = np.NaN
        
    else: 
        df_anw = pd.DataFrame(data[2:], columns=[
            'fantasy_points', 
            'city_q_fail', 
            'city_q_time',
            'city_f_fail', 
            'city_f_time',
            'stage1_fail', 
            'stage1_time',
            'stage2_fail', 
            'stage2_time',
            'stage3_fail',
            'stage4_fail'
            ])
        
    # add year column
    df_anw['year'] = year
    
    # add contestant column
    df_contestants = pd.DataFrame(data_contestants[2:], columns=['contestant'])
    
    df_combined = df_anw.join(df_contestants, how = "outer")
   
    # append the year's results to existing dataframe
    df = pd.concat([df, df_combined])

df.tail()

In [ ]:
df.info()

## Clean contestant obstacle failure data

Let's lowercase everything. 

In [ ]:
string_cols = list(df.select_dtypes(include=['object']).columns)
string_cols

for col in string_cols:
    df[col] = df[col].str.lower()
df.head()

In [ ]:
df.info()

## Initial Observations
Finally, time to have a look around. Initial question:

How many people are repeat contestants (over years 4 to 10)?

In [ ]:
df.contestant.value_counts()

Let's make the *fantasy_points* column numeric so we can do more meaningful things with it.

In [ ]:
df['fantasy_points'] = pd.to_numeric(df['fantasy_points'])
df.info()

Quite a few folks are repeaters. Let's make a histogram by count with the pandas matplotlib API.

In [ ]:
df.contestant.value_counts().hist()
plt.title("Few Contestants Were in Multiple of these Competitions")
plt.ylabel("Contestants")
plt.xlabel("Competitions")

Let's make the same chart in Seaborn just for fun and improved visualization.

In [ ]:
counters = df.contestant.value_counts()
sns.distplot(counters, kde=False)
plt.title("Few Contestants Were in Multiple of these Competitions")
plt.ylabel("Contestants")
plt.xlabel("Competitions")

Looks like maybe a slight improvement in clarity. 

We can see that the vast majority of folks who competed over these seven seasons only competed in one season. Very few folks are out at the tail of this positively skewed distribution.

Let's group by individual counts and see what the count data look like for the leaders.

In [ ]:
df_cons = df.groupby(['contestant']).count()

In [ ]:
df_cons.head()

Sort the 

In [ ]:
df_cons_sorted = df_cons.sort_values(by="fantasy_points", ascending=False)
df_cons_sorted.head(20)

Looks like almost no one makes it to stage 4. Strange there isn't a time column for stage 3. 

## Fantasy Points

What are those fantasy points anyway? According to the [website](http://www.anwfantasy.com/how-to-play/), points are scored as follows.

    1 point for every obstacle cleared!

    2 points for City Qualifier course clear
    4 points for City Finals course clears
    4 points for Midoriyama/Las Vegas Stage 1 clears
    6 points for Midoriyama/Las Vegas Stage 2 clears
    8 points for Midoriyama/Las Vegas Stage 3 clears
    10 points for Total Victory
    Additonally a 0.5 point bonus is awarded to the fastest runner for each timed stage

    For example Geoff Britten's "Perfect Season" is scored:
    • 2pts for City Qualifier cleared + 6 obstacles cleared +
    • 4pts for City Finals cleared + 10 obstacles cleared (+0.5 fastest bonus) +
    • 4pts for Stage 1 cleared + 8 obstacles cleared +
    • 6pts for Stage 2 cleared + 6 obstacles cleared (+0.5 fastest bonus) +
    • 8pts for Stage 3 cleared + 8 obstacles cleared +
    • 10pts for Stage 4 cleared + 1 obstacle cleared =
    74 total points!

Interesting. So there's a small time component in those scores, but mostly its about clearing obstacles and stages.

Let's total the contestant's fantasy points over all time and sort to see who has the most. We need to reset the index so we keep the fantasy_points as a column.

In [ ]:
df_fantasy = df.groupby(['contestant']).sum().sort_values(by="fantasy_points", ascending=False).reset_index()
df.head()

Let's drop the year column that comes along for the groupby ride.

In [ ]:
df_fantasy.drop('year', axis='columns', inplace=True)

In [ ]:
df_fantasy.head(10)

In [ ]:
df_fantasy.info()

Let's make a little chart of the top 20 folks.

In [ ]:
df_fantasy['fantasy_points'] = pd.to_numeric(df_fantasy['fantasy_points'])
df_fantasy.info()

In [ ]:
sns.catplot(x='contestant', y='fantasy_points', data=df_fantasy.head(40))
plt.title("Overall Fantasy Point Winners")
plt.ylabel("Points")
plt.xlabel("Contestant")

It's a bit tricky to read the names of top 40 fantasy scorers, eh? The plotly library does a nicer job of auto-formatting the text. Let's rotate the name vertically.

In [ ]:
sns.catplot(x='contestant', y='fantasy_points', data=df_fantasy.head(40))
plt.title("Overall Fantasy Point Winners")
plt.ylabel("Points")
plt.xlabel("Contestant")
plt.xticks(rotation=-90)

Better. But we still need a wider figure.

In [ ]:
sns.catplot(x='contestant', y='fantasy_points', data=df_fantasy.head(40), height=8, aspect=11/8)
plt.title("Overall Fantasy Point Winners")
plt.ylabel("Points")
plt.xlabel("Contestant")
plt.xticks(rotation=-90)

Ah, much nicer. And we should capitalize the first letter of each name too. The Pandas string method *capitalize* is what we want.

In [ ]:
df_fantasy['contestant'] = df_fantasy['contestant'].str.title()

In [ ]:
sns.catplot(x='contestant', y='fantasy_points', data=df_fantasy.head(40), height=8, aspect=11/8)
plt.title("Overall Fantasy Point Winners")
plt.ylabel("Points")
plt.xlabel("Contestant")
plt.xticks(rotation=-90)

## Wrap and Future Directions

In this Juptyer notebook you saw how to scrape data from a website and read it into a Pandas DataFrame in tidy data format. You saw how to manipulate the data to get information out of it and how to visualize the data.

There are several ways you could extend this project.

You could try to use historical fantasy points to predict future performance. 

Maybe a decrease in fantasy points over time tells us a compeitor is unlikely to advance further in future rounds. You could feature engineer a column for change in fantasy points over time.

It might also be interesting to try to predict contestant success with more contestant data. For example, it would be great to have additional demographic data (e.g. age, height, weight, career) and athletic performance data (e.g. vertical jump, max squat, 40yd sprint time), but I don't believe such data exists. 

Were the contestants who made it through obstacles the fastest more likely to go further in the competition? 

Finally, it might be fruitful to combine obstacle information with contestant data to try to predict contestant success.

## If you found this Kernel intersting or helpful please upvote it so that other people can find it too.